In [1]:
%env WANDB_NOTEBOOK_NAME Train.ipynb
%env CUDA_DEVICE_ORDER PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES 1
%env PYTORCH_CUDA_ALLOC_CONF backend:cudaMallocAsync

env: WANDB_NOTEBOOK_NAME=Train.ipynb
env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1
env: PYTORCH_CUDA_ALLOC_CONF=backend:cudaMallocAsync


In [2]:
from mp_20_utils import load_all_data
from tokenization import MASK_SITE
device = 'cuda'
datasets_pd, torch_datasets, site_to_ids, element_to_ids, spacegroup_to_ids, max_len = load_all_data(
    device=device, allow_retokenize=False)

In [3]:
from itertools import chain
from operator import itemgetter
max_enumeration = max(map(
    lambda dataframe: max(chain.from_iterable(dataframe.symmetry_sites_enumeration_padded)),
    datasets_pd.values()))
print(max_enumeration)

7


In [4]:
import torch
# UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
torch.set_float32_matmul_precision('high')
from wyckoff_transformer import WyckoffTransformerModel, WyckoffTrainer
d_hid = 170  # dimension of the feedforward network model in ``nn.TransformerEncoder``
nlayers = 3  # number of ``nn.TransformerEncoderLayer`` in ``nn.TransformerEncoder``
nhead = 1  # number of heads in ``nn.MultiheadAttention``
dropout = 0.1  # dropout probability
n_space_groups = len(spacegroup_to_ids)
# Not all 230 space groups are present in the data
model = WyckoffTransformerModel(
    n_space_groups = n_space_groups,
    n_sites = len(site_to_ids),
    n_elements = len(element_to_ids),
    max_enumeration = max_enumeration,
    d_space_groups = 16,
    d_sites = 64,
    d_species = 64,
    nhead=nhead,
    d_hid=d_hid,
    max_len=max_len,
    nlayers=nlayers,
    dropout=dropout).to(device)
#model = torch.compile(model)

In [5]:
import torch
trainer = WyckoffTrainer(model, torch_datasets, max_len,
                         site_mask=torch.tensor(site_to_ids[MASK_SITE]).to(device),
                         element_pad=torch.tensor(element_to_ids["PAD"]).to(device),
                         site_pad=torch.tensor(site_to_ids["PAD"]).to(device))
trainer.train(epochs=10000, val_period=10)

wandb: Currently logged in as: kazeev. Use `wandb login --relogin` to force relogin


wandb: Tracking run with wandb version 0.16.6


wandb: Run data is saved locally in /home/kna/WyckoffTransformer_dev/wandb/run-20240410_041320-fm6j3hkb
wandb: Run `wandb offline` to turn off syncing.


wandb: Syncing run lively-fog-170


wandb: ⭐️ View project at https://wandb.ai/kazeev/WyckoffTransformer


...

Epoch 5980 val_loss_epoch 10.729192733764648 saved to checkpoints/2024-04-10_04-13-19/best_model_params.pt


wandb:                                                                                


wandb: 
wandb: Run history:
wandb:          known_seq_len █▂▆▁▃▄▂▁▄█▁▂▅▆▅▇▇▇▃▂▅▃▁▃▆█▄▇▄▇▂▆▄▆▅▇▇▇▆▄
wandb:                     lr ██▇▇▆▅▄▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:       train_loss_batch ▁▅▁█▁▁▃▇▁▁▅▃▁▁▁▁▁▁▁▂▁▁▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:     train_loss_element ▁▅▁█▁▁▃▇▁▁▅▃▁▁▁▁▁▁▁▂▁▁▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb: train_loss_enumeration ▁█▁█▁▁▃▆▁▁▄▃▁▁▁▁▁▁▁▂▁▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:        train_loss_site ▁▆▁█▁▁▃▆▁▁▄▂▁▁▁▁▁▁▁▂▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:         val_loss_epoch █▆▅▄▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb: 
wandb: Run summary:
wandb:          known_seq_len 8
wandb:                     lr 1e-05
wandb:       train_loss_batch 2116.37988
wandb:     train_loss_element 1618.10791
wandb: train_loss_enumeration 199.36154
wandb:        train_loss_site 298.91043
wandb:         val_loss_epoch 10.73508
wandb: 


wandb: 🚀 View run lively-fog-170 at: https://wandb.ai/kazeev/WyckoffTransformer/runs/fm6j3hkb
wandb: ⭐️ View project at: https://wandb.ai/kazeev/WyckoffTransformer
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 1 other file(s)


wandb: Find logs at: ./wandb/run-20240410_041320-fm6j3hkb/logs
